In [17]:
from PIL import Image, ImageDraw, ImageFont
import markdown
from io import BytesIO
import requests
from bs4 import BeautifulSoup

def create_slide(title, content, image_url=None, image_subtitle=None, width=1920, height=1080):
    # Create a blank white image
    slide = Image.new('RGB', (width, height), color='white')
    draw = ImageDraw.Draw(slide)
    # Load fonts
    title_font = ImageFont.truetype("Arial", 60)
    content_font = ImageFont.truetype("Arial", 30)
    subtitle_font = ImageFont.truetype("Arial", 24)
    
    # Draw title with more padding
    draw.text((100, 100), title, font=title_font, fill='black')
    
    # Parse and draw content (Markdown)
    html = markdown.markdown(content)
    soup = BeautifulSoup(html, 'html.parser')
    
    content_width = width // 2 - 200  # Reduce content width for more padding
    y_offset = 200  # Increase initial y_offset for more top padding
    
    # Split content by double newlines
    content_blocks = content.split('\n\n')
    
    for block in content_blocks:
        block_html = markdown.markdown(block)
        block_soup = BeautifulSoup(block_html, 'html.parser')
        
        for element in block_soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'ul', 'ol']):
            if element.name == 'p':
                lines = wrap_text(element.text, content_font, content_width)
                for line in lines:
                    draw.text((100, y_offset), line, font=content_font, fill='black')
                    y_offset += 40
            elif element.name.startswith('h'):
                size = 50 - int(element.name[1]) * 5  # Adjust size based on heading level
                header_font = ImageFont.truetype("Arial", size)
                draw.text((100, y_offset), element.text, font=header_font, fill='black')
                y_offset += size + 30  # Increase space after heading
            elif element.name in ['ul', 'ol']:
                for li in element.find_all('li'):
                    bullet_lines = wrap_text('• ' + li.text, content_font, content_width - 40)
                    for line in bullet_lines:
                        draw.text((140, y_offset), line, font=content_font, fill='black')
                        y_offset += 40
        
        y_offset += 60  # Add extra space between content blocks
    
    # Add image if provided
    if image_url:
        try:
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))
            img_width = width // 2 - 200  # Reduce image width for more padding
            img_height = height - 400  # Reduce image height for more padding
            img.thumbnail((img_width, img_height))
            slide.paste(img, (width // 2 + 100, 200))  # Adjust image position
            
            if image_subtitle:
                draw.text((width // 2 + 100, height - 150), image_subtitle, font=subtitle_font, fill='black')
        except Exception as e:
            print(f"Error loading image: {e}")
    
    return slide

def wrap_text(text, font, max_width):
    words = text.split()
    lines = []
    current_line = words[0]
    
    for word in words[1:]:
        if font.getlength(current_line + ' ' + word) <= max_width:
            current_line += ' ' + word
        else:
            lines.append(current_line)
            current_line = word
    
    lines.append(current_line)
    return lines

# Example usage
title = "My Presentation Slide"

content = """
# Main Point
- Bullet point 1
- Bullet point 2


## Subheading
This is a paragraph with **bold** and *italic* text.
"""

image_url = "https://pbs.twimg.com/media/GY_lZAzbAAEGB_H?format=jpg"
image_subtitle = "Figure 1: Example Image"

slide = create_slide(title, content, image_url, image_subtitle)
slide.save("output_slide.png")